In [1]:
import time
import json
import codecs
from semanticscholar import SemanticScholar

sch = SemanticScholar(api_key="mEwrkvDmRc9KFXu76cAAk16KJxHNNUuf9tYaot1g")

In [9]:
def get_introduction(line_json):
    section_headers = json.loads(line_json["content"]["annotations"]["sectionheader"])
    start_index = next((i["start"] for i in section_headers if "attributes" in i), 0)
    end_index = next((i["end"] for i in section_headers if "attributes" in i and i["start"] > start_index), 0)
    
    paragraphs = json.loads(line_json["content"]["annotations"]["paragraph"])
    corpus = line_json["content"]["text"]
    
    introduction_para = [corpus[i["start"]:i["end"]] for i in paragraphs if start_index <= i["start"] <= end_index]
    return "\n".join(introduction_para)

id_to_introduction = {}
with codecs.open("../ScientificWritingDataset/s2orc_4000.json", 'r', encoding='utf-16') as fdata:
    for count, line in enumerate(fdata, start=1):
        try:
            line_json = json.loads(line)
            introduction = get_introduction(line_json)
            id_to_introduction[line_json["externalids"]["doi"]] = introduction
        except Exception as e:
            print(f"Failed: {count} - {e}")

paper_id_list = list(id_to_introduction.keys())

Failed: 36 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 61 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 83 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 157 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 183 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 184 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 185 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 199 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 206 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 210 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 252 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 253 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 257 - the JSON object must be str, bytes or bytearray, not NoneType
Failed: 297 - t

In [12]:
import multiprocessing
from functools import partial
from tqdm import tqdm

def process_paper_batch(paper_id_batch, id_to_introduction, sch):
    paper_json = {}
    try:
        results = sch.get_papers(paper_id_batch)
        for j, paper in enumerate(results):
            paper_json[paper["paperId"]] = {
                "paper_id": paper["paperId"],
                "title": paper["title"],
                "abstract": paper["abstract"],
                "introduction": id_to_introduction[paper_id_batch[j]],
                "fields_of_study": paper["fieldsOfStudy"],
                "publication_date": paper["publicationDate"],
                "year": paper["year"],
                "authors": paper["authors"]
            }
    except Exception as e:
        print(f"Failed: {paper_id_batch[0]} - {e}")
    return paper_json

def process_author_batch(author_id_batch, author_dict, sch):
    author_json = {}
    try:
        results = sch.get_authors(author_id_batch)
        for author in results:
            author_json[author.authorId] = {
                "id": author.authorId,
                "name": author_dict[author.authorId],
                "papers": [
                    {
                        "title": paper.title,
                        "abstract": paper.abstract,
                        "year": paper.year,
                        "fields_of_study": paper.fieldsOfStudy
                    }
                    for paper in author.papers if paper.abstract is not None
                ],
                "paper_count": author["paperCount"],
                "citation_count": author["citationCount"],
                "h_index": author["hIndex"],
                "homepage": author["homepage"]
            }
    except Exception as e:
        print(f"Failed: {author_id_batch[0]} - {e}")
    return author_json

pool = multiprocessing.Pool()
batch_size = 4

paper_batches = [paper_id_list[i:i+batch_size] for i in range(0, len(paper_id_list), batch_size)]
process_paper_batch_partial = partial(process_paper_batch, id_to_introduction=id_to_introduction, sch=sch)

paper_results = []
with tqdm(total=len(paper_batches), desc="Processing paper batches") as pbar:
    for result in pool.imap_unordered(process_paper_batch_partial, paper_batches):
        paper_results.append(result)
        pbar.update(1)

paper_json = {}
for result in paper_results:
    paper_json.update(result)


import json
import random

papers = list(paper_json.values())

random.seed(42)
random.shuffle(papers)

test_papers = papers[:500]
val_papers = papers[500:1000]
train_papers = papers[1000:]


Processing paper batches:   0%|          | 2/857 [00:31<3:04:00, 12.91s/it]

Failed: 10.1186/s13075-018-1539-6 - argument of type 'NoneType' is not iterable


Processing paper batches:   1%|          | 9/857 [00:31<19:29,  1.38s/it]  

Failed: 10.48550/arxiv.2212.04858 - argument of type 'NoneType' is not iterable


Processing paper batches:   1%|▏         | 12/857 [00:31<11:15,  1.25it/s]

...

Processing paper batches: 100%|█████████▉| 854/857 [13:56<00:01,  1.80it/s]

Failed: 10.3390/ijerph20043193 - argument of type 'NoneType' is not iterable


Processing paper batches: 100%|█████████▉| 856/857 [13:56<00:00,  2.22it/s]

Failed: 10.1016/j.ijscr.2015.05.006 - 


Processing paper batches: 100%|██████████| 857/857 [14:26<00:00,  1.01s/it]


In [13]:
author_dict = {}
author_count = sum(len(paper["authors"]) for paper in paper_json.values())
for paper in paper_json.values():
    for author in paper["authors"]:
        author_dict[author["authorId"]] = author["name"]

author_batches = [list(author_dict.keys())[i:i+batch_size] for i in range(0, len(author_dict), batch_size)]
process_author_batch_partial = partial(process_author_batch, author_dict=author_dict, sch=sch)

author_results = []
with tqdm(total=len(author_batches), desc="Processing author batches") as pbar:
    for result in pool.imap_unordered(process_author_batch_partial, author_batches):
        author_results.append(result)
        pbar.update(1)

pool.close()
pool.join()

author_json = {}
for result in author_results:
    author_json.update(result)

Processing author batches: 100%|██████████| 4116/4116 [05:00<00:00, 13.69it/s]


Failed: 150343713 - argument of type 'NoneType' is not iterable
Failed: 2149487473 - argument of type 'NoneType' is not iterable
Failed: 1887019 - Network error communicating with endpointFailed: 152132860 - Network error communicating with endpointFailed: 2150239105 - Network error communicating with endpoint


Failed: 1421847403 - Network error communicating with endpoint
Failed: 6935326 - Network error communicating with endpointFailed: 152220338 - Network error communicating with endpoint

Failed: 144927906 - Network error communicating with endpointFailed: 1392526593 - Network error communicating with endpointFailed: 2216343356 - Network error communicating with endpoint

Failed: 152138072 - Network error communicating with endpoint

Failed: 2158007852 - Network error communicating with endpoint
Failed: 2424349 - Network error communicating with endpoint
Failed: 1380784749 - Network error communicating with endpointFailed: 2095560 - Network error communicating with endpoint

Faile

Number of papers: 2744
Average authors per paper: 6.051749271137027
Number of unique authors: 16461


In [14]:
print(f"Number of papers: {len(paper_json)}")
print(f"Average authors per paper: {author_count/len(paper_json)}")
print(f"Number of unique authors: {len(author_dict)}")
print(f"Average number of papers per author: {sum(len(author['papers']) for author in author_json.values())/len(author_json)}")

Number of papers: 2744
Average authors per paper: 6.051749271137027
Number of unique authors: 16461
Average number of papers per author: 53.471269787807344


In [16]:
print(f"Title Length (Train): {sum(len(paper['title']) for paper in train_papers)/len(train_papers)}")
print(f"Abstract Length (Train): {sum(len(paper['abstract'] if paper['abstract'] else '') for paper in train_papers)/len(train_papers)}")

print(f"Title Length (Val): {sum(len(paper['title']) for paper in val_papers)/len(val_papers)}")
print(f"Abstract Length (Val): {sum(len(paper['abstract'] if paper['abstract'] else '') for paper in val_papers)/len(val_papers)}")

print(f"Title Length (Test): {sum(len(paper['title']) for paper in test_papers)/len(test_papers)}")
print(f"Abstract Length (Test): {sum(len(paper['abstract'] if paper['abstract'] else '') for paper in test_papers)/len(test_papers)}")

Title Length (Train): 97.06995412844037


TypeError: object of type 'int' has no len()

In [15]:
author_dict = {}
author_json_filtered = {}
for i in author_json:
    if len(author_json[i]["papers"])>0:
        author_dict[i] = author_json[i] 
        author_json_filtered[i]= author_json[i] 
        # print(author_json[i])
        
        # break


paperJsonFiltered = {}
author_dict_missing = {}
for i in paperJsonSampled:
    flag = True
    for author in paperJsonSampled[i]["authors"]:
        if author["authorId"] not in author_dict:
            flag= False
            author_dict_missing[author["authorId"]] = author["name"]
    if flag:
        paperJsonFiltered[i] = paperJsonSampled[i]
print(len(author_json_filtered))
print(len(author_dict_missing))
print(len(paperJsonFiltered))

with open('data_set_v2.1/author_json.json', 'w', encoding='utf-8') as f:
    json.dump(author_json_filtered, f, indent=4)
with open('data_set_v2.1/paper_json.json', 'w', encoding='utf-8') as f:
    json.dump(paperJsonFiltered, f, indent=4)

3535
82
605


In [17]:
import json
with open('data_set_v2.1/paper_json.json', 'r',encoding='utf-8') as f:
    paperJson = json.load(f)
print(len(paperJson))
with open('data_set_v2.1/author_json.json', 'r',encoding='utf-8') as f:
    authorJson = json.load(f)
print(len(authorJson))

paperDictId = {i:paperJson[i]["year"] for i in paperJson}
paperDictIdSorted = {k: v for k, v in sorted(paperDictId.items(), key=lambda item: item[1], reverse=True)}
count = 0

paperJsonSampled = {}
authorJsonSampled = {}
for i in paperDictIdSorted:
    if count < 100:
        count = count + 1
        paperJsonSampled[i] = paperJson[i]
        for author in paperJsonSampled[i]["authors"]:
            authorJsonSampled[author["authorId"]] = authorJson[author["authorId"]]
        # print(paperDictIdSorted[i])
print(len(paperJsonSampled),len(authorJsonSampled))

with open('data_set_v2.1/author_json_sampled_100.json', 'w', encoding='utf-8') as f:
    json.dump(authorJsonSampled, f, indent=4)
with open('data_set_v2.1/paper_json_sampled_100.json', 'w', encoding='utf-8') as f:
    json.dump(paperJsonSampled, f, indent=4)

605
3535
100 536
